In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import torch.optim as optim

In [2]:
file = open("C:/Users/kshitij/Desktop/NLP/DinoName/names.txt")
names = file.read().split('\n')

for i in range(len(names)):
    names[i]=names[i].lower()
#     names[i]+='.'

# names

In [3]:
alphabets = ['.','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
one = list(np.zeros((len(alphabets))))
alpha_dict={}
for n,i in enumerate(alphabets):
    one[n]=1
    alpha_dict[i] = one
    one = list(np.zeros((len(alphabets))))
# print(alpha_dict)

In [4]:
#name_one_hot_encodings
name_list = []
for i in names:
    name_encoding = []
    for j in i:
        name_encoding.append(alpha_dict[j])
    name_list.append(name_encoding)
        
for i in range(len(name_list)):
    name_list[i]=torch.Tensor(name_list[i])

In [6]:
file = open("C:/Users/kshitij/Desktop/NLP/DinoName/names.txt")
names = file.read().split('\n')

for i in range(len(names)):
    names[i]=names[i].lower()
    names[i]+='.'
print(names)

['aachenosaurus.', 'aardonyx.', 'abdallahsaurus.', 'abelisaurus.', 'abrictosaurus.', 'abrosaurus.', 'abydosaurus.', 'acanthopholis.', 'achelousaurus.', 'acheroraptor.', 'achillesaurus.', 'achillobator.', 'acristavus.', 'acrocanthosaurus.', 'acrotholus.', 'actiosaurus.', 'adamantisaurus.', 'adasaurus.', 'adelolophus.', 'adeopapposaurus.', 'aegyptosaurus.', 'aeolosaurus.', 'aepisaurus.', 'aepyornithomimus.', 'aerosteon.', 'aetonyxafromimus.', 'afrovenator.', 'agathaumas.', 'aggiosaurus.', 'agilisaurus.', 'agnosphitys.', 'agrosaurus.', 'agujaceratops.', 'agustinia.', 'ahshislepelta.', 'airakoraptor.', 'ajancingenia.', 'ajkaceratops.', 'alamosaurus.', 'alaskacephale.', 'albalophosaurus.', 'albertaceratops.', 'albertadromeus.', 'albertavenator.', 'albertonykus.', 'albertosaurus.', 'albinykus.', 'albisaurus.', 'alcovasaurus.', 'alectrosaurus.', 'aletopelta.', 'algoasaurus.', 'alioramus.', 'aliwalia.', 'allosaurus.', 'almas.', 'alnashetri.', 'alocodon.', 'altirhinus.', 'altispinax.', 'alvarez

In [7]:
#target_encodings
alpha = {'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,'n':14,'o':15,'p':16,'q':17,'r':18,'s':19,'t':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26,'.':0}
target_list = []
for i in names:
    target_encoding = []
    for j in i[1:]:
        target_encoding.append(alpha[j])
    target_list.append(target_encoding)

for i in range(len(target_list)):
    target_list[i]=torch.Tensor(target_list[i]).long()

In [24]:
class RNN(nn.Module):

    def __init__(self,input_size, hidden_size, output_size):
        self.hidden_size = hidden_size
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size,batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, output_size)
      
    def forward(self, x, h0, c0):
        x,a = self.lstm(x, (h0, c0))
        h, c = a
        x = torch.tanh(x)
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        return x,(h,c)        
        

    def initHidden(self):
        return (torch.zeros(1,1,self.hidden_size),torch.zeros(1,1,self.hidden_size))
    



In [25]:
input_size = 27
hidden_size = 50
output_size = 27
learning_rate = 0.001
model = RNN(27,50,27)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
print(model)

RNN(
  (lstm): LSTM(27, 50, batch_first=True)
  (fc1): Linear(in_features=50, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=27, bias=True)
)


In [28]:
for epochs in range(3):
    e=0
    for i in range(len(name_list)):
        optimizer.zero_grad()
        model.zero_grad()
        h,c = model.initHidden()
        x = name_list[i]
        x = x.unsqueeze(0)
        y = target_list[i]
        out,_ = model.forward(x,h,c)
        out = out.view(-1,output_size)
        loss = criterion(out, y)
        loss.backward()
        e+=loss.item()
        optimizer.step()
    print(epochs+1,' : ',e/len(name_list))

1  :  1.4798109440210585
2  :  1.457504674481849
3  :  1.4365583572071046


In [29]:
alpha = {'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,'n':14,'o':15,'p':16,'q':17,'r':18,'s':19,'t':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26,'.':0}
letter = ['.','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
digits = list(alpha.values())
with torch.no_grad():
    for n,i in enumerate(alpha_dict):
        h,c = model.initHidden()
        print()
        lis=[]
        lis.append(n)
        x = torch.Tensor(alpha_dict[i])
        x = x.view(1,1,-1)
        output,state = model.forward(x,h,c)
        h, c = state
        output = F.softmax(output,dim=2)
        et = torch.argmax(output)
#         print(et)
        x = torch.zeros(x.shape)
        x[0,0,et] = 1
        while et!=0:
            output,state = model.forward(x,h,c)
            h,c = state
            output = F.softmax(output,dim=2)
            et = torch.argmax(output)
            x = torch.zeros(x.shape)
            x[0,0,et] = 1
            lis.append(et)
#         print(lis)
        for k in lis:
            print(letter[k],end='')
        


.ngosaurus.
agangosaurus.
broceratops.
cenosaurus.
dinosaurus.
eatosaurus.
flangosaurus.
gngosaurus.
hngong.
iosaurus.
jangangosaurus.
kltong.
lnosaurus.
mnosaurus.
npelton.
onithongx.
poconosaurus.
qangnosaurus.
rptoraptor.
sronosaurus.
tngangosaurus.
uanosaurus.
vnopelta.
wlanosaurus.
xngosaurus.
yngangosaurus.
zongosaurus.